In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 26 1:21:17 2021

@author: tasfi
"""
import matplotlib
matplotlib.use("Agg")
from matplotlib import colors, ticker, patches
from matplotlib import patheffects, gridspec
import matplotlib.dates as mdates
import numpy as np
import matplotlib.pyplot as plt
import iris # minimal fio done here 
import iris.quickplot as qplt
import cartopy.crs as ccrs
from datetime import datetime, timedelta
import numpy as np
from netCDF4 import Dataset
#from oceans import wrap_lon180
import oceans
import mplleaflet
#import Leaflet as L
import pandas as pd 
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import branca.colormap
from collections import defaultdict
import webbrowser


Bad key "text.kerning_factor" on line 4 in
C:\Users\tshermin\AppData\Local\Continuum\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
firei = iris.load("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/firefront.20200102T1500Z.nc")
#print(wind_u)
print(np.max(firei[0].data))
print(np.min(firei[0].data))
print('\n')
#print(wind_v)
fireii = firei.extract("firefront")[0] 
fireI = fireii[0]
print(np.min(fireii.data))
print(np.max(fireii.data))

0.00839999970048666
-0.050337099538912265


-0.050337099538912265
0.00839999970048666


In [3]:

from matplotlib import patheffects, colors, image, ticker

#create map
map0 = folium.Map(location=[-35.5, 136.4],#tiles="Cartodb Positron",
                  zoom_start=9)

#legend
steps=33
colormap = branca.colormap.LinearColormap(['indigo','blue','lightblue',
                                           'cyan','green','darkgreen',
                                           'yellow','orange','red','magenta',
                                           'purple','pink','white','black','black','black'],
                               index=[0.0,0.03,0.06,0.09,0.12,0.15,
                                      0.18,0.21,0.24,0.27,0.30,0.33,0.36,.4,.6,.8])
colormap._repr_html_()



colormap1 = branca.colormap.LinearColormap(colors=['cyan','yellow','BlueViolet',
                                                   'lime','magenta','orange'
                                                   ,'red','green','indigo',
                                                  'brown'],
                                          index=[0,3,6,9,12,15,18,21,24,27],vmin=0,vmax=27
                                            ,caption='Horizontal Wind Speed')#.to_step(steps)

colormap0 = branca.colormap.LinearColormap(colors=['Plum','violet','DarkOrchid',
                                                   'fuchsia','MediumOrchid','MediumPurple'
                                                   ,'BlueViolet','DarkMagenta','indigo',
                                                  'MediumSlateBlue'],
                                          index=[0,3,6,9,12,15,18,21,24,27],vmin=0,vmax=27
                                            ,caption='Horizontal Wind Speed')#.to_step(steps)
        
    
#gradient_dict = {}

step = branca.colormap.StepColormap(
 colors=['indigo','blue','lightblue','cyan','green','lightgreen','yellow','orange','red','magenta','purple','pink','white'],                                          
 vmin=0, vmax=360,
 index=[0,30,60,90,120,150,180,210,240,270,300,330,360],  #for change in the colors, not used fr linear
 caption='Color Scale for Map'    #Caption for Color scale or Legend
)

#colormap.add_to(map0) 
#colormap1.add_to(map0) 
#step.add_to(map0)      
#linear.add_to(map0)
#branca.colormap.linear

end =[]
tmp =[]
tmp2 = [] 
time = 480
#x_wind0 = wind_U.extract("UWIND_2")[0]
#y_wind0 = wind_V.extract("VWIND_2")[0]
fireii = firei.extract("firefront")[0] 
for i in range(time):
    if i%10==0:

        fireI = fireii[i]
        #print(fireI.data.shape)

        lats,lons = fireI.coord('latitude').points, fireI.coord('longitude').points

    
        #u = x_wind
        #v = y_wind
        lon = lons
        lat = lats
        lat, lon = np.meshgrid(lat, lon)
        mask_x = np.logical_and(lon > 136, lon < 138)
    #print(mask_x)
        mask_y = np.logical_and(lat > -38, lat < -35)
    #print(mask_y)
        mask = np.logical_and(mask_x, mask_y)
    #print(mask)
    
        x = lon[mask]
        y = lat[mask]
    
        F = fireI.data[mask]#.filled(fill_value=0)
        
        #if np.any(fireii[i].data < 0):
        #if np.any(F.data < 0):
            #for j in range(8640):
                #if F[j]<0:
                    #F[j] = (6 + F[j])*0.1
                    #print(F[j])
                #else:
                    #F[j] = (1 - F[j])*0.01
                #out = np.array([y[j], x[j], F[j]]).tolist()
                ##print(F[j])
                #end.append(out)           
                #tmp.append(out)
                #end=[]
            
            #tmp2.append(tmp)
            #tmp = []
        for j in range(8640):
            if F[j]<0:
                F[j] = (6 + F[j])*0.1
                    #print(F[j])
            else:
                F[j] = (1 - F[j])*0.01
            out = np.array([y[j], x[j], F[j]]).tolist()
                #print(F[j])
            end.append(out)           
            tmp.append(out)
            end=[]
        tmp2.append(tmp)
        tmp = []
                
    #if np.any(fireI.data < 0):
        #out = np.array([lats, lons, fireI.data.T]).tolist()
        #end.append(out)           
        #tmp.append(out)
        #end=[]
        
    
        #plt.contour(lons,lats,ffti,np.array([0]),transform=ccrs.PlateCarree(),colors='red')
    
        
    #print(i)
#print(tmp2)

heatmap = folium.plugins.HeatMapWithTime(tmp2,radius=20,#11,#12,#15, 
                                          min_opacity=0.5, max_opacity=25, 
                                          scale_radius=False,
                                          gradient= {'0.0':'lightblue','.49':'lightsalmon','.5991':'coral','.5994':'tomato',
                                                     '.5997':'orangered','.5999':'red','0.9':'red'},
                                         #gradient= {'0.0':'Plum','0.1':'violet','0.2':'darkOrchid',
                                                   #'0.3':'fuchsia','0.4':'mediumOrchid','0.5':'mediumPurple '
                                                   #,'0.6':'blueViolet','0.7':'darkMagenta','0.8':'indigo',
                                                  #'0.9':'MediumSlateBlue'},
                                              
                                            
                                          #0.0,0.03,0.06,0.09,0.12,0.15,0.18,0.21,0.24,0.27,0.30,0.33,0.36
                                          #gradient=gradient_map1,
                                          #use_local_extrema=False, 
                                         auto_play=False, 
                                          #display_index=True, index_steps=1, min_speed=0.1,
                                          #max_speed=10, speed_step=0.1, position='bottomleft', 
                                          #overlay=True, control=True, show=True
                                        )
    
    
map0.add_child(heatmap)
    

map0.add_child(folium.map.LayerControl())


map0.save (r'C:\PHD\APR-BOM\visualize\my codes\exploratory_KI_run1\fire\firefrontwithTime_superzoomed.html')
webbrowser.open(r'C:\PHD\APR-BOM\visualize\my codes\exploratory_KI_run1\fire\firefrontwithTime_superzoomed.html')


True